# 11. Magentic Workflow Agent Workshop

이 노트북에서는 Microsoft Agent Framework의 Magentic 워크플로우를 사용하여 복합 에이전트 시스템을 구축하는 방법을 학습합니다.

## 학습 목표
- Magentic 워크플로우의 개념과 구조 이해
- 다중 에이전트 오케스트레이션 구현
- 워크플로우를 에이전트로 래핑하여 재사용하는 방법
- 스트리밍 이벤트 처리 및 콜백 구현

## 시나리오
연구자와 코더 에이전트가 협력하여 머신러닝 모델의 에너지 효율성을 분석하는 워크플로우를 구축합니다.

## 1. 필수 라이브러리 및 모듈 임포트

In [18]:
# Microsoft Agent Framework의 Magentic 워크플로우 관련 모듈들
import asyncio
import logging
import os
from typing import cast
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv(override=True)

from agent_framework import (
    MAGENTIC_EVENT_TYPE_AGENT_DELTA,
    MAGENTIC_EVENT_TYPE_ORCHESTRATOR,
    AgentRunUpdateEvent,
    ChatAgent,
    ChatMessage,
    HostedCodeInterpreterTool,
    MagenticBuilder,
    WorkflowOutputEvent,
)
# Azure OpenAI 클라이언트 (Azure 환경에서 사용)
from agent_framework.azure import AzureOpenAIChatClient

# 로깅 설정
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

print("라이브러리 임포트 완료!")
print("✅ AgentRunUpdateEvent 및 이벤트 타입 상수 임포트됨")

라이브러리 임포트 완료!
✅ AgentRunUpdateEvent 및 이벤트 타입 상수 임포트됨


## 2. Azure OpenAI 설정

Magentic 워크플로우에서 사용할 Azure OpenAI 클라이언트를 설정합니다.
- `AzureOpenAIChatClient`: Azure OpenAI 서비스를 통한 채팅 및 분석 작업용
- 환경변수를 통한 Azure OpenAI 연결 설정

In [8]:
# 환경 변수 확인
print("환경 변수 확인:")
env_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY", 
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME",
    "AZURE_OPENAI_API_KEY"
]

for var in env_vars:
    value = os.getenv(var)
    if value:
        # API 키는 부분적으로만 표시
        if "API_KEY" in var:
            print(f"✅ {var}: {value[:8]}...")
        else:
            print(f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: 설정되지 않음")

print("\n💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.")

환경 변수 확인:
✅ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✅ AZURE_OPENAI_API_KEY: 6hBhIb88...
✅ AZURE_OPENAI_API_VERSION: 2025-04-01-preview
✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1
✅ AZURE_OPENAI_API_KEY: 6hBhIb88...

💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.


### Azure OpenAI 설정 예제

Azure OpenAI를 사용하기 위해 다음 환경변수들을 `.env` 파일에 설정하세요:

```bash
# .env 파일 예제
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key-here
AZURE_OPENAI_API_VERSION=2024-06-01
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4o
```

## 3. 에이전트 생성

두 개의 전문 에이전트를 생성합니다:
1. **Researcher Agent**: 연구 및 정보 수집 전문
2. **Coder Agent**: 코드 작성 및 데이터 분석 전문

In [9]:
# 연구자 에이전트 생성
# Official sample처럼 OpenAIChatClient 사용 (웹 검색 기능 포함)
# Azure를 사용하려면 아래 주석 참조
researcher_agent = ChatAgent(
    name="ResearcherAgent",
    description="Specialist in research and information gathering",
    instructions=(
        "You are a Researcher. You find information without additional "
        "computation or quantitative analysis. Focus on gathering factual "
        "information, research papers, and established knowledge."
    ),
    # OpenAI ChatClient 사용 (Official sample 권장)
    # 웹 검색 기능이 필요한 경우: model_id="gpt-4o-search-preview"
    # chat_client=OpenAIChatClient(model_id="gpt-4o"),
    
    # Azure OpenAI를 사용하려면 아래처럼 변경:
    chat_client=AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    ),
)

print("✅ ResearcherAgent 생성 완료")

# 코더 에이전트 생성 (코드 인터프리터 기능 포함)
# Azure OpenAI로 코드 실행 지원
coder_agent = ChatAgent(
    name="CoderAgent",
    description="A helpful assistant that writes and executes code to process and analyze data.",
    instructions=(
        "You solve questions using code. Please provide detailed analysis "
        "and computation process. Write clean, well-documented code."
    ),
    # Azure OpenAI ChatClient + 코드 인터프리터
    chat_client=AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    ),
    tools=HostedCodeInterpreterTool(),
)

print("✅ CoderAgent 생성 완료")
print("두 에이전트가 성공적으로 생성되었습니다!")

✅ ResearcherAgent 생성 완료
✅ CoderAgent 생성 완료
두 에이전트가 성공적으로 생성되었습니다!


## 4. Magentic 워크플로우 구축

MagenticBuilder를 사용하여 다중 에이전트 워크플로우를 구성합니다:
- **Participants**: 워크플로우에 참여할 에이전트들 정의
- **Manager**: 에이전트 간 조정 및 관리 설정

In [10]:
print("🔧 Magentic 워크플로우 구축 중...")

# Manager 에이전트 생성 (Official sample 패턴, Azure OpenAI 사용)
manager_agent = ChatAgent(
    name="MagenticManager",
    description="Orchestrator that coordinates the research and coding workflow",
    instructions="You coordinate a team to complete complex tasks efficiently.",
    chat_client=AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    ),
)

# MagenticBuilder를 사용하여 워크플로우 구성 (Official sample 패턴)
workflow = (
    MagenticBuilder()
    .participants(researcher=researcher_agent, coder=coder_agent)
    .with_standard_manager(
        agent=manager_agent,        # manager agent 전달
        max_round_count=10,         # 최대 대화 라운드 수
        max_stall_count=3,          # 진행 정체 최대 허용 횟수
        max_reset_count=2,          # 워크플로우 재시작 최대 횟수
    )
    .build()
)

print("✅ Magentic 워크플로우가 성공적으로 구축되었습니다!")
print(f"참여 에이전트: {len([researcher_agent, coder_agent])}개")
print("워크플로우 매니저 설정 완료")

🔧 Magentic 워크플로우 구축 중...
✅ Magentic 워크플로우가 성공적으로 구축되었습니다!
참여 에이전트: 2개
워크플로우 매니저 설정 완료


## 5. 분석 작업 정의

머신러닝 모델의 에너지 효율성을 비교 분석하는 복합적인 작업을 정의합니다.

In [ ]:
# 분석 작업 정의 - 머신러닝 모델 에너지 효율성 비교
task = (
    "다양한 머신러닝 모델 아키텍처의 에너지 효율성에 관한 보고서를 준비 중입니다. "
    # "Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 "
    # "on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). "
    # "Then, estimate the CO2 emissions associated with each, assuming training on an Azure Standard_NC6s_v3 "
    # "VM for 24 hours. Provide tables for clarity, and recommend the most energy-efficient model "
    # "per task type (image classification, text classification, and text generation)."
)

print("📋 분석 작업:")
print("=" * 60)
print(task)
print("=" * 60)
print("\n이 작업은 다음과 같은 단계로 수행됩니다:")
print("1. 연구자 에이전트: 각 모델의 에너지 소비 데이터 조사")
print("2. 코더 에이전트: 수집된 데이터를 기반으로 계산 및 비교 분석")
print("3. 협업: 최종 보고서 및 권장사항 작성")

📋 분석 작업:
I am preparing a report on the energy efficiency of different machine learning model architectures. Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). Then, estimate the CO2 emissions associated with each, assuming training on an Azure Standard_NC6s_v3 VM for 24 hours. Provide tables for clarity, and recommend the most energy-efficient model per task type (image classification, text classification, and text generation).

이 작업은 다음과 같은 단계로 수행됩니다:
1. 연구자 에이전트: 각 모델의 에너지 소비 데이터 조사
2. 코더 에이전트: 수집된 데이터를 기반으로 계산 및 비교 분석
3. 협업: 최종 보고서 및 권장사항 작성


## 6. 워크플로우 실행 (스트리밍)

워크플로우를 실행하고 실시간 스트리밍 이벤트를 처리합니다. 
각 에이전트의 작업 진행 상황과 결과를 실시간으로 확인할 수 있습니다.

In [12]:
async def run_workflow_with_streaming():
    """워크플로우를 실행하고 스트리밍 이벤트 처리 (Official sample 패턴)"""
    
    print("🚀 워크플로우 실행 시작...")
    print("=" * 70)
    
    try:
        # 스트리밍 상태 관리 변수
        last_stream_agent_id: str | None = None
        stream_line_open: bool = False
        output: str | None = None
        
        # 워크플로우 실행 및 스트리밍 이벤트 처리 (Official sample 패턴)
        async for event in workflow.run_stream(task):
            
            # AgentRunUpdateEvent로 통합된 이벤트 처리
            if isinstance(event, AgentRunUpdateEvent):
                # additional_properties에서 이벤트 타입 확인
                props = event.data.additional_properties if event.data else None
                event_type = props.get("magentic_event_type") if props else None
                
                # 오케스트레이터 메시지 처리
                if event_type == MAGENTIC_EVENT_TYPE_ORCHESTRATOR:
                    kind = props.get("orchestrator_message_kind", "") if props else ""
                    text = event.data.text if event.data else ""
                    print(f"\n[ORCH:{kind}]\n\n{text}\n{'-' * 26}")
                
                # 에이전트 델타 (스트리밍) 처리
                elif event_type == MAGENTIC_EVENT_TYPE_AGENT_DELTA:
                    agent_id = props.get("agent_id", event.executor_id) if props else event.executor_id
                    
                    if last_stream_agent_id != agent_id or not stream_line_open:
                        if stream_line_open:
                            print()
                        print(f"\n[STREAM:{agent_id}]: ", end="", flush=True)
                        last_stream_agent_id = agent_id
                        stream_line_open = True
                    
                    if event.data and event.data.text:
                        print(event.data.text, end="", flush=True)
                
                # 기타 이벤트 처리
                elif event.data and event.data.text:
                    print(event.data.text, end="", flush=True)
            
            # 워크플로우 출력 이벤트
            elif isinstance(event, WorkflowOutputEvent):
                output_messages = cast(list[ChatMessage], event.data)
                if output_messages:
                    output = output_messages[-1].text
        
        # 스트리밍 정리
        if stream_line_open:
            print()
            stream_line_open = False
        
        if output is not None:
            print(f"\n워크플로우 완료:\n\n{output}")
        
        return output
    
    except Exception as e:
        print(f"워크플로우 실행 실패: {e}")
        return None

# 데모 실행을 위한 간단한 버전 (실제 API 키가 있을 때만 실행)
print("워크플로우 스트리밍 함수가 정의되었습니다.")
print("✅ Official sample 패턴으로 이벤트 처리 구현")
print("실제 실행하려면 다음 셀을 실행하세요 (OpenAI API 키 필요).")

워크플로우 스트리밍 함수가 정의되었습니다.
✅ Official sample 패턴으로 이벤트 처리 구현
실제 실행하려면 다음 셀을 실행하세요 (OpenAI API 키 필요).


In [13]:
# 워크플로우 실행 (Azure OpenAI 또는 OpenAI API 키가 설정된 경우에만 실행)
# 주석을 해제하고 실행하세요

# Azure OpenAI 또는 OpenAI 설정 확인
azure_configured = all([
    os.getenv("AZURE_OPENAI_ENDPOINT"),
    os.getenv("AZURE_OPENAI_API_KEY"),
    os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
])
openai_configured = bool(os.getenv("OPENAI_API_KEY"))

if azure_configured or openai_configured:
    result = await run_workflow_with_streaming()
    print(f"\n워크플로우 실행 결과: {result}")
else:
    print("⚠️  워크플로우 실행을 위해서는 Azure OpenAI 또는 OpenAI API 설정이 필요합니다.")
    print("환경변수를 설정한 후 다시 실행해주세요.")

print("💡 워크플로우 실행 준비 완료!")
print("실제 실행을 위해서는 위의 주석을 해제하고 Azure OpenAI 또는 OpenAI API를 설정하세요.")

🚀 워크플로우 실행 시작...

[ORCH:user_task]

I am preparing a report on the energy efficiency of different machine learning model architectures. Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). Then, estimate the CO2 emissions associated with each, assuming training on an Azure Standard_NC6s_v3 VM for 24 hours. Provide tables for clarity, and recommend the most energy-efficient model per task type (image classification, text classification, and text generation).
--------------------------

[ORCH:task_ledger]


We are working to address the following user request:

I am preparing a report on the energy efficiency of different machine learning model architectures. Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). Then, estimate the CO

## 7. 워크플로우를 에이전트로 래핑

Magentic의 강력한 기능 중 하나는 전체 워크플로우를 단일 에이전트로 래핑하여 다른 시스템에서 재사용할 수 있다는 점입니다.

In [ ]:
async def demo_workflow_as_agent():
    """워크플로우를 에이전트로 래핑하여 사용하는 예제"""
    
    print("🔄 워크플로우를 에이전트로 래핑...")
    
    # 워크플로우를 단일 에이전트로 래핑
    workflow_agent = workflow.as_agent(name="MagenticWorkflowAgent")
    
    print("✅ 워크플로우 에이전트 생성 완료!")
    print(f"에이전트 이름: {workflow_agent}")
    
    # 간단한 테스트 작업
    simple_task = "기계 학습 모델의 에너지 효율성에 영향을 미치는 주요 요인을 요약하십시오."
    
    print(f"\n📝 테스트 작업: {simple_task}")
    
    try:
        # 래핑된 에이전트로 작업 실행
        agent_result = await workflow_agent.run(simple_task)
        
        if agent_result.messages:
            print("\n" + "=" * 60)
            print("📋 에이전트 실행 결과 (대화 기록)")
            print("=" * 60)
            
            for i, msg in enumerate(agent_result.messages, start=1):
                role_value = getattr(msg.role, "value", msg.role)
                speaker = msg.author_name or role_value
                print(f"\n{'-' * 50}")
                print(f"{i:02d} [{speaker}]")
                print(f"{msg.text}")
        
        return agent_result
    
    except Exception as e:
        print(f"❌ 에이전트 실행 실패: {e}")
        return None

# 워크플로우 에이전트 래핑 함수 정의 완료
print("📦 워크플로우 에이전트 래핑 함수가 정의되었습니다.")
print("실제 실행하려면 다음 셀을 실행하세요.")

📦 워크플로우 에이전트 래핑 함수가 정의되었습니다.
실제 실행하려면 다음 셀을 실행하세요.


In [15]:
# 워크플로우 에이전트 데모 실행 (Azure OpenAI 또는 OpenAI API 키가 필요)
# 주석을 해제하고 실행하세요

# Azure OpenAI 또는 OpenAI 설정 확인
azure_configured = all([
    os.getenv("AZURE_OPENAI_ENDPOINT"),
    os.getenv("AZURE_OPENAI_API_KEY"),
    os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
])
openai_configured = bool(os.getenv("OPENAI_API_KEY"))

if azure_configured or openai_configured:
    agent_result = await demo_workflow_as_agent()
    if agent_result:
        print("\n🎉 워크플로우 에이전트 실행 성공!")
        # agent_result에서 결과값 출력
        if agent_result.messages:
            print("\n📋 에이전트 실행 결과 (요약):")
            for i, msg in enumerate(agent_result.messages, 1):
                role_value = getattr(msg.role, "value", msg.role)
                speaker = msg.author_name or role_value
                text = (msg.text or "").strip()
                print(f"{i:02d} [{speaker}]: {text[:300]}{'...' if len(text) > 300 else ''}")
        else:
            print("결과 메시지가 없습니다.")
else:
    print("⚠️  워크플로우 에이전트 실행을 위해서는 Azure OpenAI 또는 OpenAI API 설정이 필요합니다.")

print("🎯 워크플로우 에이전트 데모 준비 완료!")
print("이제 복잡한 다중 에이전트 워크플로우를 단일 에이전트처럼 사용할 수 있습니다.")

🔄 워크플로우를 에이전트로 래핑...
✅ 워크플로우 에이전트 생성 완료!
에이전트 이름: <agent_framework._workflows._agent.WorkflowAgent object at 0x116263680>

📝 테스트 작업: Summarize the key factors that affect machine learning model energy efficiency.

📋 에이전트 실행 결과 (대화 기록)

--------------------------------------------------
01 [ResearcherAgent]
Certainly! Here is a clear, organized summary of the **key factors affecting energy efficiency in machine learning model training and inference**, with authoritative references.

---

## Key Factors Affecting ML Model Energy Efficiency

### 1. **Model Architecture and Size**
- **Complexity:** Larger models (more parameters/layers) require more computation, leading to higher energy consumption.
    - *Example:* GPT-3 consumes orders of magnitude more energy than BERT-base due to its size ([Patterson et al., 2021](https://arxiv.org/abs/2104.10375)).
- **Type and Depth:** Deep convolutional networks (e.g., ResNet) and transformer-based models (e.g., BERT, GPT) differ in computational de

## 8. 고급 기능 및 설정

Magentic 워크플로우의 고급 기능들을 살펴보겠습니다.

In [16]:
# Magentic 워크플로우의 고급 설정들

print("🔧 Magentic 워크플로우 고급 설정 가이드")
print("=" * 60)

# 1. 매니저 설정 매개변수들
print("\n1. 워크플로우 매니저 설정:")
print("   - max_round_count: 최대 대화 라운드 (기본값: 10)")
print("   - max_stall_count: 진행 정체 허용 횟수 (기본값: 3)")
print("   - max_reset_count: 워크플로우 재시작 횟수 (기본값: 2)")

# 2. 이벤트 타입들 (Official sample 패턴)
print("\n2. 스트리밍 이벤트 타입들 (Official sample 패턴):")
print("   - AgentRunUpdateEvent: 통합된 에이전트 실행 이벤트")
print("     • MAGENTIC_EVENT_TYPE_ORCHESTRATOR: 오케스트레이터의 조정 메시지")
print("     • MAGENTIC_EVENT_TYPE_AGENT_DELTA: 에이전트의 실시간 응답 스트림")
print("   - WorkflowOutputEvent: 워크플로우 출력 데이터")
print("")
print("   💡 Official sample은 event.data.additional_properties에서")
print("      magentic_event_type을 확인하여 이벤트를 구분합니다.")

# 3. 에이전트 도구들
print("\n3. 사용 가능한 에이전트 도구들:")
tools_info = {
    "HostedCodeInterpreterTool": "코드 실행 및 분석",
    "WebSearchTool": "웹 검색 (모델 의존적)",
    "FileAccessTool": "파일 읽기/쓰기 작업",
    "DatabaseTool": "데이터베이스 연동"
}

for tool, description in tools_info.items():
    print(f"   - {tool}: {description}")

print("\n4. 모범 사례 (Official sample 기준):")
print("   ✅ OpenAIChatClient/OpenAIResponsesClient 사용 권장")
print("   ✅ Manager를 별도 ChatAgent로 생성 (with_standard_manager에 전달)")
print("   ✅ 에이전트별로 명확한 역할과 지시사항 정의")
print("   ✅ AgentRunUpdateEvent로 통합된 이벤트 처리")
print("   ✅ 적절한 모델 선택 (검색: gpt-4o-search-preview, 코드: gpt-4o)")
print("   ✅ 스트리밍 이벤트 적절히 처리하여 사용자 경험 향상")
print("   ✅ 워크플로우를 에이전트로 래핑하여 재사용성 향상")

🔧 Magentic 워크플로우 고급 설정 가이드

1. 워크플로우 매니저 설정:
   - max_round_count: 최대 대화 라운드 (기본값: 10)
   - max_stall_count: 진행 정체 허용 횟수 (기본값: 3)
   - max_reset_count: 워크플로우 재시작 횟수 (기본값: 2)

2. 스트리밍 이벤트 타입들 (Official sample 패턴):
   - AgentRunUpdateEvent: 통합된 에이전트 실행 이벤트
     • MAGENTIC_EVENT_TYPE_ORCHESTRATOR: 오케스트레이터의 조정 메시지
     • MAGENTIC_EVENT_TYPE_AGENT_DELTA: 에이전트의 실시간 응답 스트림
   - WorkflowOutputEvent: 워크플로우 출력 데이터

   💡 Official sample은 event.data.additional_properties에서
      magentic_event_type을 확인하여 이벤트를 구분합니다.

3. 사용 가능한 에이전트 도구들:
   - HostedCodeInterpreterTool: 코드 실행 및 분석
   - WebSearchTool: 웹 검색 (모델 의존적)
   - FileAccessTool: 파일 읽기/쓰기 작업
   - DatabaseTool: 데이터베이스 연동

4. 모범 사례 (Official sample 기준):
   ✅ OpenAIChatClient/OpenAIResponsesClient 사용 권장
   ✅ Manager를 별도 ChatAgent로 생성 (with_standard_manager에 전달)
   ✅ 에이전트별로 명확한 역할과 지시사항 정의
   ✅ AgentRunUpdateEvent로 통합된 이벤트 처리
   ✅ 적절한 모델 선택 (검색: gpt-4o-search-preview, 코드: gpt-4o)
   ✅ 스트리밍 이벤트 적절히 처리하여 사용자 경험 향상
   ✅ 워크플로우를 에이전트로 래핑하여 재사용성 향상


## 9. 실습 과제 및 확장

다음과 같은 실습을 통해 Magentic 워크플로우를 더 깊이 이해해보세요.

In [17]:
print("🎯 Magentic 워크플로우 실습 과제")
print("=" * 50)

exercises = {
    "초급": [
        "다른 도메인의 전문 에이전트 추가 (예: 디자이너, 마케터)",
        "워크플로우 매니저 설정 값 조정하여 성능 비교",
        "다른 OpenAI 모델들로 에이전트 구성해보기"
    ],
    "중급": [
        "커스텀 도구를 가진 에이전트 생성",
        "조건부 워크플로우 로직 구현",
        "에러 처리 및 재시도 메커니즘 강화"
    ],
    "고급": [
        "동적으로 에이전트를 추가/제거하는 워크플로우",
        "외부 API와 연동하는 에이전트 개발",
        "복수의 워크플로우를 연계하는 메타 워크플로우"
    ]
}

for level, tasks in exercises.items():
    print(f"\n{level} 실습:")
    for i, task in enumerate(tasks, 1):
        print(f"   {i}. {task}")

print("\n" + "=" * 50)
print("🚀 확장 아이디어:")
print("   • 실시간 협업 플랫폼 구축")
print("   • 자동화된 리서치 파이프라인")
print("   • 코드 리뷰 및 개선 워크플로우")
print("   • 멀티모달 에이전트 (텍스트 + 이미지 + 음성)")
print("   • 장기 메모리를 가진 지속적 학습 에이전트")

print("\n💡 다음 단계:")
print("   1. OpenAI API 키 설정 후 실제 워크플로우 실행")
print("   2. 다양한 에이전트 조합으로 실험")
print("   3. 실제 비즈니스 문제에 적용해보기")
print("   4. 성능 모니터링 및 최적화")

🎯 Magentic 워크플로우 실습 과제

초급 실습:
   1. 다른 도메인의 전문 에이전트 추가 (예: 디자이너, 마케터)
   2. 워크플로우 매니저 설정 값 조정하여 성능 비교
   3. 다른 OpenAI 모델들로 에이전트 구성해보기

중급 실습:
   1. 커스텀 도구를 가진 에이전트 생성
   2. 조건부 워크플로우 로직 구현
   3. 에러 처리 및 재시도 메커니즘 강화

고급 실습:
   1. 동적으로 에이전트를 추가/제거하는 워크플로우
   2. 외부 API와 연동하는 에이전트 개발
   3. 복수의 워크플로우를 연계하는 메타 워크플로우

🚀 확장 아이디어:
   • 실시간 협업 플랫폼 구축
   • 자동화된 리서치 파이프라인
   • 코드 리뷰 및 개선 워크플로우
   • 멀티모달 에이전트 (텍스트 + 이미지 + 음성)
   • 장기 메모리를 가진 지속적 학습 에이전트

💡 다음 단계:
   1. OpenAI API 키 설정 후 실제 워크플로우 실행
   2. 다양한 에이전트 조합으로 실험
   3. 실제 비즈니스 문제에 적용해보기
   4. 성능 모니터링 및 최적화


## 요약 및 핵심 포인트

### Magentic 워크플로우의 핵심 개념

1. **다중 에이전트 오케스트레이션**: 여러 전문 에이전트가 협력하여 복잡한 작업 수행
2. **스트리밍 이벤트**: 실시간으로 워크플로우 진행 상황 모니터링
3. **에이전트 래핑**: 워크플로우 전체를 단일 에이전트로 재사용 가능
4. **유연한 설정**: 매니저 설정을 통한 워크플로우 동작 제어

### 주요 장점
- ✅ **모듈성**: 각 에이전트가 독립적인 전문 영역 담당
- ✅ **확장성**: 새로운 에이전트 쉽게 추가 가능
- ✅ **재사용성**: as_agent()를 통한 워크플로우 재활용
- ✅ **투명성**: 스트리밍을 통한 실시간 진행 상황 파악

### Official Sample 패턴 (microsoft/agent-framework)
본 노트북은 공식 샘플을 기반으로 리팩토링되었습니다:
- 🔹 **OpenAI 클라이언트 사용**: OpenAIChatClient, OpenAIResponsesClient
- 🔹 **Manager Agent 패턴**: 별도 ChatAgent를 생성하여 with_standard_manager에 전달
- 🔹 **통합 이벤트 처리**: AgentRunUpdateEvent + additional_properties로 이벤트 타입 구분
- 🔹 **이벤트 타입 상수**: MAGENTIC_EVENT_TYPE_ORCHESTRATOR, MAGENTIC_EVENT_TYPE_AGENT_DELTA

### 활용 사례
- 📊 **연구 및 분석**: 정보 수집 + 데이터 분석 + 보고서 작성
- 🤖 **소프트웨어 개발**: 요구사항 분석 + 코드 생성 + 테스트 + 리뷰
- 📈 **비즈니스 자동화**: 데이터 수집 + 분석 + 의사결정 지원
- 🎓 **교육 도구**: 질문 분석 + 학습자료 검색 + 맞춤형 설명 생성

### Azure OpenAI 사용법
본 노트북은 OpenAI를 기본으로 하되, Azure OpenAI로 전환하려면:
```python
# Azure OpenAI 클라이언트로 변경
chat_client=AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-06-01",
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
)
```

이 워크샵을 통해 Microsoft Agent Framework의 Magentic 워크플로우를 활용한 지능형 에이전트 시스템 구축 방법을 학습했습니다!